## Installing DKube SDK

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/oneconvergence/dkube.git@2.2-dev --upgrade --no-cache-dir > /dev/null

# Importing DKube SDK

In [ ]:
from dkube.sdk import *

# Input and output parameters for the run

In [ ]:
user = os.getenv('USERNAME')
authToken = os.getenv('DKUBE_USER_ACCESS_TOKEN')
project_id = "ynli7c"
project_name = "titanic"
ptrain_dataset = 'titanic-train'
ptest_dataset = 'titanic-test'
dataset_mount_points = ["/opt/dkube/input/train", "/opt/dkube/input/test"]
ptrain_out_path = "/opt/dkube/output/train"
ptest_out_path = "/opt/dkube/output/test"
train_fs = "titanic-train-fs"
test_fs = "titanic-test-fs"
model_name = "titanic"
framework = "scikit_v0.2.2"
image = "docker.io/ocdr/dkube-datascience-tf-cpu:fs-v2.0.0"
preprocess_script = f"python preprocessing.py --train_fs {train_fs} --test_fs {test_fs}"
training_script = f"python training.py --train_fs {train_fs}"

# DKube client handle

In [ ]:
api = DkubeApi(token=authToken)

# Updating preprocessing run details

In [ ]:
preprocessing_name= generate(project_name)
preprocessing = DkubePreprocessing(user, name=preprocessing_name, description='triggered from dkube sdk', tags = ["project:" + project_id])
preprocessing.update_container(image_url=image)
preprocessing.update_startupscript(preprocess_script)
preprocessing.add_project(project_name)
preprocessing.add_input_dataset(ptrain_dataset, mountpath=dataset_mount_points[0])
preprocessing.add_input_dataset(ptest_dataset, mountpath=dataset_mount_points[1])
preprocessing.add_output_featureset(train_fs, mountpath=ptrain_out_path)
preprocessing.add_output_featureset(test_fs, mountpath=ptest_out_path)

# Creating preprocessing run

In [ ]:
# Skip if preprocessing is not required
api.create_preprocessing_run(preprocessing)

# Updating training run details

In [ ]:
training_name = generate(project_name)
training = DkubeTraining(user, name = training_name, description='triggered from dkube sdk', tags = ["project:" + project_id])
training.update_container(framework = framework, image_url=image)
training.update_startupscript(training_script)
training.add_project(project_name)
training.add_input_featureset(train_fs, mountpath="/titanic-train")
training.add_input_featureset(test_fs, mountpath="/titanic-test")
training.add_output_model(model_name, mountpath="/model")

# Creating training run

In [ ]:
api.create_training_run(training)